## Trabajar con tipos de datos de fecha y hora en Amazon DynamoDB

DynamoDB no tiene un tipo de datos de fecha y hora dedicado. En este artículo, mostraremos cómo consultar datos de fecha y hora a través de un caso de uso de cámaras de tráfico colocadas en una autopista muy transitada para registrar la velocidad de los coches con el fin de identificar los vehículos que superan el límite de velocidad (exceso de velocidad). Utilizaremos los modelos datetime y timestamp para recuperar datos de tablas.

#### ISO 8601
Puede almacenar datos de fecha y hora como un tipo de datos numérico o de cadena en DynamoDB. Para los datos de fecha y hora almacenados como cadena, siga la norma ISO 8601. ISO 8601 es una norma internacional que cubre el intercambio y la comunicación a nivel mundial de datos relacionados con la fecha y la hora.

ISO 8601 representa la fecha y la hora empezando por el año, seguido del mes, el día, la hora, los minutos, los segundos y los milisegundos.

* **2022-01-10** - Se trata de una fecha de calendario sin hora, que indica el 10 de enero de 2022.

* **2022-09-15T08** - Es una fecha de calendario con hora. La hora está separada de la fecha por una T. Este ejemplo se interpreta como las 8:00 AM del 15 de septiembre de 2022.
Cuando no se especifica la hora, se supone que la hora es 00:00 en la zona horaria local. ISO 8601 admite otros formatos de fecha y hora.

#### Trabajar con datos datetime
La tabla de prueba para utilizar datetime tiene los atributos `ID` (ID de cámara), `DT` (fecha y hora), `Location`, `Vehicle#` (número de vehículo) y `VehicleSpeed` (velocidad del vehículo). ID es la clave de partición y DT es la clave de ordenación, que se combinan para actuar como clave primaria compuesta.

In [1]:
import boto3
import pandas as pd
from botocore.exceptions import ClientError
from spdynamodb import DynamoTable
import json
from decimal import Decimal
from datetime import datetime, timedelta
import time
import random

In [2]:
dt=DynamoTable()
try:
    dt.select_table('SampleTable')
    print(dt)
except:
    dt.create_table(
        table_name='SampleTable',
        partition_key='ID',
        partition_key_type='S',
        sort_key='DT',
        sort_key_type='S',
    )

- Table name: SampleTable            
- Table arn: arn:aws:dynamodb:us-east-1:203645649179:table/SampleTable            
- Table creation: 2023-08-02 09:04:18            
- [{'AttributeName': 'ID', 'KeyType': 'HASH'}, {'AttributeName': 'DT', 'KeyType': 'RANGE'}]            
- [{'AttributeName': 'DT', 'AttributeType': 'S'}, {'AttributeName': 'ID', 'AttributeType': 'S'}]            
- Point-in-time recovery status: DISABLED  |  Delete protection: False


In [8]:
samples = 200
id_data = ['A1', 'A2', 'A3', 'A4', 'A5']
location = ['Street A', 'Street B', 'Street C', 'Street D', 'Street E']
vehicle = ['PT-9754-MA', 'MT-9754-MA', 'XS-9754-MA', 'PT-9754-MA', 'MT-9754-MA', 'XS-9754-MA']
speed = random.sample(range(10, 100), 20)
all_items = []

for i in range(samples):
    month = random.choice(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'])
    day = random.choice(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'])
    year = random.choice(['2019', '2020', '2021', '2022', '2023'])
    random_let = random.choice(['A', 'B', 'C'])
    if random_let == 'A':
        dt_data = f'{year}-{month}T10:07:22'
    if random_let == 'B':
        dt_data = f'{year}-{month}-{day}T18:08:22'
    else:
        dt_data = f'{year}-{month}-{day}'
    
    Item={
            'ID': random.choice(id_data),
            'DT': dt_data,
            'Location': random.choice(location),
            'Vehicle#': random.choice(vehicle),
            'VehicleSpeed': random.choice(speed)
    }
    all_items.append(Item)

In [9]:
# Save to json file
with open('timeserie_sample.json', 'w') as outfile:
    json.dump(all_items, outfile, indent=4)

# Write to DynamoDB table
dt.load_json('timeserie_sample.json')

Data loaded successfully from timeserie_sample.json.


En apoyo del caso de uso, tiene sentido mantener la fecha y la hora como clave de ordenación hasta el nivel granular de segundos. Por ejemplo, la cámara con ID A2 puede grabar un vehículo el 13 de julio de 2022 a las 9:23:45 y la misma cámara puede capturar otro vehículo en la misma fecha pero a las 9:23:47, creando así dos registros únicos.

Desde el punto de vista de la lógica empresarial, esto tiene sentido, ya que cada cámara tiene asignado un ID único y, utilizando los metadatos de la cámara, se puede conocer su ubicación. La fecha y la hora se utilizan para determinar cuándo se creó un registro. A continuación, puede consultar la tabla DynamoDB para extraer información como:
* El número de vehículos registrados en o después de una fecha específica, por ejemplo, el 12 de julio de 2022, que excedieron el límite de velocidad.
* El número de vehículos registrados con exceso de velocidad en un mes específico.

Utilizar la fecha y la hora como clave principal no es eficiente, ya que aumentaría la cardinalidad y el número de particiones necesarias para almacenar físicamente los datos. Algunos días podrían tener más tráfico que otros, creando así particiones desequilibradas.

Recuperar los datos del vehículo para la cámara A1 después del 12 de julio de 2022. La clave de ordenación de la tabla es DT (DateTime) representada como un tipo de datos de cadena.

In [12]:
dt.query_partiql("SELECT * FROM {} WHERE ID = 'A1' AND DT > '2022-07-12'".format(dt.table_name), to_pandas=True)

,Location,DT,VehicleSpeed,ID,Vehicle#
0,Street B,2022-08-08T18:08:22,89,A1,XS-9754-MA
1,Street E,2022-09-09,26,A1,XS-9754-MA
2,Street C,2022-10-07,64,A1,XS-9754-MA
3,Street D,2023-04-01,26,A1,XS-9754-MA
4,Street A,2023-07-09,26,A1,PT-9754-MA
5,Street A,2023-09-05,13,A1,PT-9754-MA


Recuperar los detalles del vehículo captados por la cámara A2 antes del 13 de enero de 2022, a las 18:10:24.

In [18]:
dt.query_partiql("SELECT * FROM {} WHERE ID = 'A2' AND DT < '2020-01-13T18:10:24'".format(dt.table_name), to_pandas=True, consumed_capacity=True)

Consumed Capacity: 0.5


,Location,DT,VehicleSpeed,ID,Vehicle#
0,Street C,2019-02-10T18:08:22,40,A2,PT-9754-MA
1,Street A,2019-03-01,14,A2,MT-9754-MA
2,Street A,2019-04-11T18:08:22,22,A2,MT-9754-MA
3,Street D,2019-07-08,98,A2,MT-9754-MA
4,Street E,2019-07-11T18:08:22,24,A2,PT-9754-MA
5,Street C,2019-09-05T18:08:22,50,A2,XS-9754-MA
6,Street A,2019-12-09,55,A2,XS-9754-MA
7,Street C,2020-01-03,61,A2,XS-9754-MA


Recuperar los detalles del vehículo para el mes de julio de 2022, independientemente del ID de la cámara. Ejecute el siguiente análisis:

In [20]:
dt.query_partiql("SELECT * FROM {} WHERE DT > '2022-07-01' AND DT < '2022-07-31'".format(dt.table_name), to_pandas=True)

,Location,DT,VehicleSpeed,ID,Vehicle#
0,Street E,2022-07-12T18:08:22,22,A4,MT-9754-MA
1,Street C,2022-07-07T18:08:22,26,A1,MT-9754-MA
2,Street B,2022-07-03,64,A3,PT-9754-MA


Recuperar todos los vehículos que circulen a más de 96 mph durante el año 2022.

In [29]:
dt.query_partiql("SELECT * FROM {} WHERE DT > '2021' AND DT < '2023' AND VehicleSpeed > 96".format(dt.table_name), to_pandas=True)

,Location,DT,VehicleSpeed,ID,Vehicle#
0,Street E,2021-08-02,97,A2,PT-9754-MA
1,Street A,2021-08-09,98,A2,MT-9754-MA
2,Street E,2021-09-02,98,A2,MT-9754-MA
3,Street E,2022-12-05T18:08:22,98,A2,PT-9754-MA
4,Street B,2021-11-09,98,A5,PT-9754-MA
5,Street D,2022-02-10,98,A5,PT-9754-MA
6,Street E,2021-10-10T18:08:22,97,A1,MT-9754-MA
7,Street D,2022-04-12,97,A1,XS-9754-MA
8,Street A,2021-07-09,97,A3,PT-9754-MA
9,Street D,2021-10-07T18:08:22,98,A3,PT-9754-MA


#### Trabajar con datos de marca de tiempo

Tal vez la forma más conveniente de almacenar datos de marca de tiempo sea utilizar el tipo de datos numérico para almacenar el tiempo de época, ya que la marca de tiempo es un dato numérico por característica. Una marca de tiempo Unix puede almacenarse en segundos, milisegundos, microsegundos o nanosegundos, dependiendo del nivel de precisión requerido para el caso de uso. En este ejemplo, la marca de tiempo se almacena en segundos.

Para la mayoría de los casos de uso, tiene sentido utilizar la marca de tiempo como clave de ordenación en lugar de la clave de partición porque habría demasiadas particiones con muchos menos datos si se utilizara la marca de tiempo como clave de partición.